In [111]:
import psycopg2
from psycopg2.extensions import AsIs
import psycopg2
import pandas as pd
import numpy as np
import shapely
from shapely import wkb
from shapely.geometry import mapping, shape

In [112]:

db_credentials = {"dbname": 'gta',
                  "port": 5432,
                  "user": 'gta_p8',
                  "password": 'r7sdkfdq',
                  "host": 'ikgpgis.ethz.ch'}

In [113]:
#fiktiver input von flask
trip_id = 10

In [114]:
def get_linestring(trip_id, db_credentials):
    trip_id = str(trip_id)
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()
    sql_string = "SELECT geometry FROM gta_p8.trip WHERE trip_id='"+trip_id+"';"
    cur.execute(sql_string)
    wkb_hex = cur.fetchone()
    conn.commit()
    conn.close()
    linestring = wkb.loads(bytes.fromhex(wkb_hex[0]))
    return linestring


In [115]:
def buffering_linestring(linestring, distance=15.0/1000):
    return linestring.buffer(distance)

In [116]:
def get_point_ids(buffer_polygon, db_credentials):
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()
    sql_string = "SELECT restaurant_id FROM gta_p8.restaurant WHERE ST_Contains(ST_GeomFromText('"+str(buffer_polygon)+"', 4326), geometry);"
    cur.execute(sql_string)
    list = cur.fetchone()
    return list


In [ ]:
def get_point_ids(buffer_polygon, db_credentials):
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()
    sql_string = "SELECT restaurant_id FROM gta_p8.restaurant WHERE ST_Contains(gta_p8.restaurant.geometry, geometry);"
    cur.execute(sql_string)
    list = cur.fetchone()
    return list

In [117]:
linestring = get_linestring(trip_id, db_credentials)
buffer_polygon = buffering_linestring(linestring=linestring)
print(buffer_polygon)
point_id = get_point_ids(buffer_polygon, db_credentials)

POLYGON ((8.495291533294793 47.41719525516577, 8.496158419075943 47.41832277327327, 8.497111828779319 47.41944432707717, 8.498170579050965 47.42046702980025, 8.49932447354699 47.42138103225624, 8.500562399632615 47.42217753210184, 8.501872435402918 47.42284885860812, 8.50324196449729 47.42338854653385, 8.504657797601894 47.42379139838928, 8.506106299469957 47.42405353449082, 8.50757352023666 47.4241724303245, 8.509045329763953 47.42414694085847, 8.510507553721517 47.42397731157021, 8.511946110093309 47.42366517608249, 8.51334714479507 47.4232135404307, 8.514697165096742 47.42262675411305, 8.515983169564821 47.42191046820253, 8.51719277327328 47.42107158092406, 8.518314327077167 47.42011817122068, 8.519337029800251 47.41905942094903, 8.52025103225624 47.41790552645301, 8.521047532101839 47.41666760036738, 8.521718858608125 47.41535756459708, 8.522258546533854 47.41398803550271, 8.522661398389282 47.41257220239811, 8.522923534490817 47.41112370053004, 8.523042430324505 47.40965647976334,

In [118]:
print(point_id)

(859,)


In [ ]:
#st_buffer in st_contains aufpassen mit geometry

SELECT restaurant_id 
FROM gta_p8.restaurant as R
WHERE ST_Contains((SELECT ST_Buffer(T.geometry,30) FROM gta_p8.trip as T), R.geometry)
